In [9]:
def parse_mdp(file_path):
    transitions = {}
    end_states = set()
    num_states = num_actions = 0
    mdptype = ""
    discount = 0.0

    with open(file_path, 'r') as file:
        for line in file:
            tokens = line.strip().split()
            if not tokens:
                continue
            if tokens[0] == 'numStates':
                num_states = int(tokens[1])
            elif tokens[0] == 'numActions':
                num_actions = int(tokens[1])
            elif tokens[0] == 'end':
                if len(tokens) > 1 and tokens[1] != "-1":
                    end_states = set(map(int, tokens[1:]))
            elif tokens[0] == 'transition':
                s1 = int(tokens[1])
                a = int(tokens[2])
                s2 = int(tokens[3])
                r = float(tokens[4])
                p = float(tokens[5])
                transitions.setdefault((s1, a), []).append((s2, r, p))
            elif tokens[0] == 'mdptype':
                mdptype = tokens[1]
            elif tokens[0] == 'discount':
                discount = float(tokens[1])
    return num_states, num_actions, end_states, transitions, mdptype, discount                                      

In [10]:
import numpy as np
def value_iteration(num_states,num_actions, end_states, transitions, discount, epsilon=1e-8):
    V = np.zeros(num_states)
    while True:
        delta = 0
        V_new = np.copy(V)
        for s in range(num_states):
            if s in end_states:
                continue
            q_values = []
            for a in range(num_actions):
                q = 0
                for (s_, r, p) in transitions.get((s,a), []):
                    q += p*(r + discount*V[s_])
                q_values.append(q)
            if q_values:
                V_new[s] = max(q_values)
                delta = max(delta, abs(V[s] - V_new[s]))         
        V = V_new
        if delta < epsilon:
            break            
    return V

In [11]:
def extract_policy(num_states, num_actions, end_states, transitions, V, discount):
    policy = np.zeros(num_states, dtype=int)

    for s in range(num_states):
        if s in end_states:
            policy[s] = 0
            continue
        q_values =[]
        for a in range(num_actions):
            q = 0
            for (s_, r, p) in transitions.get((s,a), []):
                q += p*(r+ discount*V[s_])
            q_values.append(q)  

        if q_values:
            policy[s] = int(np.argmax(q_values))
        else:
            policy[s] = 0
    return policy                  

In [12]:
mdp_files = [
    "data/continuing-mdp-2-2.txt",
    "data/continuing-mdp-10-5.txt",
    "data/continuing-mdp-50-20.txt",
    "data/episodic-mdp-2-2.txt",
    "data/episodic-mdp-10-5.txt",
    "data/episodic-mdp-50-20.txt"
]
for path in mdp_files:
    print(f"Solving: {path}")
    num_states, num_actions, end_states, transitions, mdptype, discount = parse_mdp(path)
    V_star = value_iteration(num_states, num_actions, end_states, transitions, discount)
    pi_star = extract_policy(num_states, num_actions, end_states, transitions, V_star, discount)

    for s in range(num_states):
        print(f"{V_star[s]:.6f} {pi_star[s]}")

Solving: data/continuing-mdp-2-2.txt
5.999299 0
5.918450 0
Solving: data/continuing-mdp-10-5.txt
2.234958 3
2.373612 3
2.604046 3
2.647784 1
2.522231 4
2.375252 0
2.684806 2
2.688310 0
2.640809 3
2.572427 1
Solving: data/continuing-mdp-50-20.txt
1.065079 7
1.051696 2
0.824259 7
0.601320 14
1.057797 4
0.980877 19
0.983041 18
1.002595 5
0.886921 15
0.837798 8
1.109280 8
0.910305 19
1.155357 7
0.958098 8
0.772395 18
1.218694 16
0.939597 11
0.840961 19
0.934034 2
0.899851 12
1.168103 14
0.985183 19
1.032489 14
1.110618 15
0.779151 0
0.945382 1
1.185461 3
1.083733 18
0.697620 15
1.125198 5
0.556266 1
1.088646 6
0.829482 11
0.884322 6
1.180251 1
0.922217 4
0.916141 11
1.031048 10
1.077761 14
0.900197 19
0.855533 5
1.205419 0
1.056961 4
0.720773 14
1.141582 1
1.110485 4
0.983264 5
1.030596 3
0.779689 1
0.815195 12
Solving: data/episodic-mdp-2-2.txt
0.000000 0
1.455816 0
Solving: data/episodic-mdp-10-5.txt
0.000000 0
530.219800 3
530.513651 4
504.798626 2
472.948045 1
0.000000 0
526.952970 2
5